# 量子计算
$$
\newcommand{\braket}[2]{\left.\left\langle#1\right|#2\right\rangle}
\newcommand{\sbraket}[1]{\braket{#1}{#1}}
\newcommand{\bracket}[3]{\left.\langle#1\right.\left|#2\right|\left.#3\right\rangle}
\newcommand{\bra}[1]{\left\langle#1\right|}
\newcommand{\ket}[1]{\left|#1\right\rangle}
\newcommand{\norm}[1]{\left\lVert#1\rVert\right}
\newcommand{\exp}[1]{\left\langle#1\right\rangle}
$$

本篇是 *UIUC PHYS 370 Introduction to Quantum Information and Computing* 的学习笔记，涵盖了量子电路、量子算法等话题。

In [14]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

## 目录

## 线性代数

### 矢量与狄拉克记号

量子力学建立于线性代数之上；本篇笔记中我们会大量使用线性代数中的概念，如矢量、矩阵、特征值等；它们在量子力学中具有特定的物理意义。由于物理学家采用了有别于数学家的记号，即使是对线性代数非常了解的同学也推荐快速浏览一下本章节。

**矢量（Vector）** 是 **矢量空间（Vector Space）** 中的量。从初等的角度可以将它们理解成带有方向的量，或空间中的一个点，与仅有大小的 **标量（Scalar）** 相对。从抽象的角度，则定义矢量空间为定义了矢量加法：$+: V\times V \to V$ 和标量乘法 $\cdot: V\times F \to V$ 的集合 $V(F)$，且满足：

- 标量集合 $F$ 是一个数域，即定义了封闭的加减乘除运算。本篇笔记中出现的矢量空间一律 $F=\mathbb{C}$。
- 矢量加法和标量乘法均封闭。
- 矢量加法满足交换律、结合律，并存在零矢量 $0$ 使得 $v + 0 = 0 + v = v$ 对于任意 $v \in V$ 成立。
- 标量乘法满足交换律、结合律、和分配律：$a(u + v) = au + av$ 且 $(a + b)v = av + bv$ 对于任意 $a, b \in F, u, v \in V$ 成立。

数学中通常不对矢量进行任何特殊记号，仅使用 $u, v, w$ 等末位拉丁字母与标量 $a, b, c$ 之类的分开；有时也会用上面带箭头的形式（如 $\vec{u}$）。物理中则会将矢量符号加粗，写成 $\mathbf{a}, \mathbf{b}$ 等样式。量子力学中使用的是 **狄拉克记号（Dirac Notation）**，或 **左右矢记号（Bra-ket Notation）**，将矢量记作 $|\psi\rangle$ 这样希腊字母套在 $|\rangle$ 符号中的形式。

值得特别一提的是，虽然在记号 $\ket{\alpha}$ 中，$\alpha$ 只是一个标签，但是在实践中它也可以表示矢量 $\ket{\alpha}$ 本身。比如笔记中可能会出现类如 $\ket{k\alpha}$ 的简记法，此时它的含义是 $k\ket{\alpha}$，而 $k\alpha$ 是这个矢量的标签。

一集矢量 $|\alpha\rangle, |\beta\rangle, |\gamma\rangle$ 称为 **线性无关的（Linearly Independent）**，如果它们的 **线性组合（Linear Combination）**（下面方程等号左侧的形式）满足：

$$
\begin{equation*}
    a|\alpha\rangle + b|\beta\rangle + c|\gamma\rangle + \dots = 0
\end{equation*}
$$

当且仅当 $a = b = c = \dots = 0$ 时成立。换句话说，这集矢量中的任一个都不能表示为其它矢量的加权和。一集矢量 **张成（Span）** 的空间定义为其所有线性组合形成的矢量空间。如果一个矢量空间 $V$ 可以被一集线性无关的矢量张成，这集向量就是这个 $V$ 的一个 **基（Basis）**，此时可以将矢量空间中任一个元素 $\ket{\alpha}$ 分解为：

$$
\begin{equation*}
    |\alpha\rangle = a_1|e_1\rangle + \dots + a_n|e_n\rangle
\end{equation*}
$$

我们发现，如果给定了矢量空间的一个基，那么任意矢量都可以通过 $n$ 个标量唯一地表示，这样就得到了矢量的另一种形式：

$$
\begin{equation*}
    |\alpha\rangle = \begin{pmatrix} a_1 \\ \vdots \\ a_n \end{pmatrix}
\end{equation*}
$$

不难从矢量空间运算的特点得到下面的结论：

$$
\begin{equation*}
    |\alpha\rangle + |\beta\rangle
        = \begin{pmatrix} a_1 + b_1 \\ \vdots \\ a_n + b_n \end{pmatrix} \qquad
    \lambda|\alpha\rangle
        = \begin{pmatrix} ca_1 \\ \vdots \\ ca_n \end{pmatrix} \qquad
    |-\alpha\rangle
        = \begin{pmatrix} -a_1 \\ \vdots \\ -a_n \end{pmatrix} \qquad
    |0\rangle
        = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix}
\end{equation*}
$$

如果矢量空间被大小为 $n$ 的基张成，则称该矢量空间的 **维度（Dimension）** 为 $n$，记作 $\text{dim}(V) = n$。可以看到这和我们在矢量分析中用到的术语含义相似。值得一提的是，对于同一个矢量空间，无论选取的基是什么，其数量总是等于矢量空间的维度；如果一集线性无关的矢量张成了一个矢量空间 $V$，则这集矢量就是 $V$ 的一个基。

至此我们介绍的矢量称为 **右矢（Ket）**，它也是和数学中经典的矢量对应的概念。下面我们将补全另一半，称为 **左矢（Bra）**。从名字中可以看出，这也是一个矢量，因此可以通过某个基用一组数唯一地表示；不过左矢的形式和右矢有所不同：

$$
\begin{equation*}
    \langle\alpha| = \begin{pmatrix} a_1^* & \dots & a_n^* \end{pmatrix}
\end{equation*}
$$

其中 $a_j^*$ 是 $a_j$ 的 **复数共轭（Complex Conjugate）**，定义为 $z^* = \frac{|z|^2}{z}$（$0$ 的共轭则为 $0$）。可以看到，左矢和右矢的形式有微妙的不同，比起后者写成一列数的形式（因此也称为 **列矢量（Column Vector）**），前者将数写在一行上（故也称为 **行矢量（Row Vector）**）。左矢和右矢不在同一个矢量空间，因此两者不能相加；不过两者所在的矢量空间互为对方的 **对偶空间（Dual Space）**，这种特殊的关系使得它们之间可以进行特别的操作，我们会在后续小节中详细说明。

和右矢类似地，左矢的标签可以代表右矢本身（注意不是左矢本身！），如 $\bra{k\alpha}$ 代表了 $k\ket{\alpha}$ 对应的左矢。

Python 中的 `numpy.array` 提供了类似与矢量的数据结构，我们本篇中将使用该类型模拟矢量：

In [46]:
def bra(*args):
    return np.array([args])

def ket(*args):
    res = []
    for arg in args:
        res.append([arg])
    return np.array(res)

def is_bra(v):
    return v.shape[0] == 1

def is_ket(v):
    return v.shape[1] == 1

# dagger 函数将 bra 和 ket 互相转换。后面我们会看到，这不仅适用于矢量
def dagger(a):
    return a.conj().T

print(bra(1+2j, 1j))
print(ket(1+2j, 1j))
print(dagger(bra(1+2j, 1j)))
print(dagger(ket(1+2j, 1j)))

[[1.+2.j 0.+1.j]]
[[1.+2.j]
 [0.+1.j]]
[[1.-2.j]
 [0.-1.j]]
[[1.-2.j 0.-1.j]]


### 线性变换和矩阵

**线性变换（Linear Transformation）** 是从一个矢量空间映射到另一个矢量空间的函数 $T: V \to W$。它需要满足下面的性质：

- $T(u + v) = T(u) + T(v)$。
- $T(av) = aT(v)$

数学中的线性变换常写成大写的拉丁字母；量子力学中，我们使用带有尖角符号的大写字母表示线性变换，如 $\hat{T}$。此外，线性变换虽然是函数，在使用时通常不用圆括号将参数扩进来，因此后面我们会使用 $\hat{T}v$ 而非 $T(v)$ 的形式。为了用更加直观的方式表示线性变换，让我们考察其连接的两个矢量空间的基的关系。设矢量空间 $V$ 的基为 $v_1, \dots, v_m$，其通过 $\hat{T}$ 映射到另一个矢量空间 $W$，后者的基为 $w_1, \dots, w_n$。根据线性变换的特征，我们可以将 $W$ 中的每个基写成下面的等式：

$$
\begin{equation*}
    w_k = T_{1k}v_1 + \dots + T_{mk}v_m
\end{equation*}
$$

其中每个 $T_{ij}, i=1,\dots,m, j=1,\dots,n$ 都是域上的线性变换（通常都是标量），其可以通过 $V$ 和 $W$ 以及它们基的选取唯一确定。综上，我们可以将线性变换 $\hat{T}$ 写成下面的形式：

$$
\begin{equation*}
    \hat{T} = \begin{pmatrix} T_{11} & \dots & T_{1n} \\ \vdots & \ddots & \vdots \\ T_{m1} & \dots & T_{mn} \end{pmatrix}
\end{equation*}
$$

这个结构称为 **矩阵（Matrix）**。更具体来说，它是一个 $m\times n$（$m$ 行 $n$ 列）的矩阵。由于线性变换和矩阵的等价性，我们此后将不区分地使用这两个词（主要使用矩阵）。观察可以发现，此前我们定义地左右矢在表示形式上和矩阵非常相似；事实正是如此：矢量可以看作矩阵的特殊情况（左矢是 $1\times n$ 的矩阵，而右矢是 $m\times 1$ 的矩阵），因此下面介绍的运算规则同样适用于矢量：

两个 $m\times n$ 矩阵的加法定义如下：

$$
\begin{equation*}
    \hat{T} + \hat{S} = \begin{pmatrix}
        T_{11} + S_{11} & \dots & T_{1n} + S_{1n} \\
        \vdots & \ddots & \vdots \\
        T_{m1} + S_{m1} & \dots & T_{mn} + S_{mn} 
    \end{pmatrix}
\end{equation*}
$$

矩阵的标量积定义如下：

$$
\begin{equation*}
    c\hat{T} = \begin{pmatrix}
        cT_{11} & \dots & cT_{1n} \\
        \vdots & \ddots & \vdots \\
        cT_{m1} & \dots & cT_{mn}
    \end{pmatrix}
\end{equation*}
$$

定义零矩阵为所有元素均为 $0$ 的矩阵：

$$
\begin{equation*}
    \hat{0} = \begin{pmatrix}
         0 & \dots & 0 \\
         \vdots & \ddots & \vdots \\
         0 & \dots & 0 
    \end{pmatrix}
\end{equation*}
$$

在上面的设置下，我们不难发现 $m\times n$ 矩阵所在的空间也是矢量空间，记作 $M_{m\times n}$，而左矢和右矢所在的矢量空间只是其中的两个特例。

对于矢量空间 $M_{m\times \ell}$ 和 $M_{\ell\times n}$，定义 **矩阵乘法（Matrix Multiplication）** 如下：

$$
\begin{equation*}
    (TU)_{ij} = \sum_{k=1}^\ell T_{ik}U_{kj}, \qquad T \in M_{m\times \ell},\ U \in M_{\ell\times n},\ i=1,\dots,m,\ j=1,\dots,n
\end{equation*}
$$

可以看到这个结果是矢量空间 $M_{m\times n}$ 中的矩阵。可以证明，矩阵乘法满足结合律（即运算的先后顺序不影响结果），但一般不满足交换律。这个运算的主要意义在于，其将线性变换抽象成独立于矢量之外的存在。比如对矢量 $|\psi\rangle$ 先后进行变换 $T$ 和 $U$，从最开始的定义来看是 $U(T|\psi\rangle)$；通过抽象出矩阵和矩阵乘法，我们也可以将这个操作理解为两个线性变换的乘积 $UT$ 代表了新的变换，并作用于矢量 $|\psi\rangle$ 得到 $(UT)|\psi\rangle$。后面我们会认识到这种灵活性的好处。

`numpy.array` 同样可以用于存储矩阵（事实上，此前我们左右矢就存成了二维的数组，即矩阵），下面我们将演示矩阵的乘法操作：

In [43]:
T = np.array([[2+1j, 3+2j], [1, 0], [-1-1j, 2j]], dtype=complex)
U = np.array([[2+3j, 1, -3-2j], [0, 3+1j, 4-3j]], dtype=complex)
v = ket(1-2j, 0, 1+2j)
# 检测结合律。注意 numpy.array 的矩阵乘法使用的是 @，而 * 是完全不相关的含义！
print((T @ U) @ v)
print(T @ (U @ v))

[[ 47.+26.j]
 [  9. -9.j]
 [-28.+20.j]]
[[ 47.+26.j]
 [  9. -9.j]
 [-28.+20.j]]


### 内积空间

**内积空间（Inner Product Space）** 是定义了 **内积（Inner Product）** 运算 $\langle\cdot|\cdot\rangle: V\times V \to F$ 的矢量空间，其需要满足下面的性质：

- 交换后互为复数共轭，即 $\langle u|v \rangle = \langle v|u \rangle^*$。
- 与自身的内积为非负实数，即 $\langle v|v \rangle \ge 0$。
- 满足线性变换的要求，即 $\langle au|v + w \rangle = a\langle u|v \rangle + a\langle u|w \rangle$。

实际上从内积的记号就可以看出，互为对偶空间中的左矢和右矢之积就是我们想要的内积：

$$
\begin{equation*}
    \langle \alpha|\beta \rangle = \langle \alpha||\beta \rangle = a_1^*b_1 + \dots + a_n^*b_n
\end{equation*}
$$

特别地，我们将矢量 $|\psi\rangle$ 的 **范数（Norm）** 定义为其与自身之积的平方根：

$$
\begin{equation*}
    \lVert{\psi}\rVert = \sqrt{\langle \psi|\psi \rangle}
\end{equation*}
$$

根据内积的定义，这个值一定是非负实数。它的几何意义正是矢量的长度。范数为 $1$ 的矢量被称为 **归一化矢量（Normalized Vector）**。如果两个矢量的内积为零，则称它们相互 **正交（Orthogonal）**。特别地，如果一集矢量两两之间均正交，则称它们为 **标准正交集（Orthonormal Set）**。我们最常见到的标准正交集构成一个矢量空间的基，因此称其为 **标准正交基（Orthonormal Basis）**。举例来说，矢量空间 $\mathbb{C}^3$ 中的 $|e_x\rangle$、$|e_y\rangle$、$|e_z\rangle$ 就是一个标准正交基。为了在计算中方便表示，我们通常将标准正交基中任两个基向量的内积写成：

$$
\begin{equation*}
    \langle e_i|e_j \rangle = \delta_{ij} = \begin{cases}
        1 & i = j \\
        0 & i \ne j
    \end{cases}
\end{equation*}
$$

这里的 $\delta_{ij}$ 称为 **克罗内克记号（Kronecker Delta）**。通过这个记号的帮助，我们可以给出矢量在标准正交基中每个基矢量方向上的分量：

$$
\begin{equation*}
    a_i = \sum_{j=0}^n \delta_{ij}a_j = \sum_{j=0}^n \langle e_i|e_j \rangle a_j = \langle e_i| \sum_{j=0}^n a_j|e_j\rangle
    = \langle e_i|\alpha \rangle
\end{equation*}
$$

熟悉矢量分析的同学可以看到它和 $a_i = \hat{e}_i\cdot\mathbf{a}$ 的一致性；不过在内积空间中，我们不能随意调换内积两侧的矢量，即 $\langle e_i|\alpha \rangle \ne \langle \alpha|e_i \rangle$。 实际上：

$$
\begin{equation*}
    \langle \alpha|e_i \rangle = \langle e_i|\alpha \rangle^* = a_j^*
\end{equation*}
$$

有两个著名的不等式和内积相关，其一是 **施瓦兹不等式（Schwarz Inequality）**：

$$
\begin{equation*}
    |\langle \alpha|\beta \rangle|^2 \le \langle \alpha|\alpha \rangle\langle \beta|\beta \rangle
\end{equation*}
$$

如果从实数的矢量空间来看，这个不等式等价于 $|\cos\theta| \le 1$。不过在复空间中，我们最好求助于代数。注意到 $|\langle \alpha|\beta \rangle|^2 = \langle \alpha|\beta \rangle^* \langle \alpha|\beta \rangle$，设：

$$
\begin{equation*}
    |\gamma\rangle = |\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}|\alpha\rangle
\end{equation*}
$$

（上面假设 $|\alpha\rangle \ne |0\rangle$，这种情况下施瓦兹不等式显然成立）考虑 $\langle \gamma|\gamma \rangle \ge 0$ 则有：

$$
\begin{align*}
    \langle \gamma|\gamma \rangle
        &= \left(\langle\beta| - \frac{\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle^*}\langle\alpha|\right)
           \left(|\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}|\alpha\rangle\right) \\
        &= \langle\beta|\beta\rangle + \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle^*\langle\alpha|\alpha\rangle}\langle\alpha|\alpha\rangle - \frac{\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle^*}\langle\alpha|\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}\langle\beta|\alpha\rangle \\
        &= \langle\beta|\beta\rangle + \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle^*} - \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha^*\rangle} - \frac{\langle\alpha|\beta\rangle\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle} \\
        &= \langle\beta|\beta\rangle - \frac{|\langle\alpha|\beta\rangle|^2}{\langle\alpha|\alpha\rangle} \\
        &\ge 0
\end{align*}
$$

由于 $\langle\alpha|\alpha\rangle > 0$，将两边同时乘以 $\langle\alpha|\alpha\rangle$ 得到 $\langle\alpha|\alpha\rangle\langle\beta|\beta\rangle \ge |\langle\alpha|\beta\rangle|^2$，得证。

另外一个不等式是 **三角不等式（Triangle Inequality）**，即：

$$
\begin{equation*}
    \left\lVert\alpha + \beta\right\rVert \le \left\lVert\alpha\right\rVert + \left\lVert\beta\right\rVert
\end{equation*}
$$

考察不等式左侧的平方：

$$
\begin{align*}
    \left\lVert\ket{\alpha} + \ket{\beta}\right\rVert^2 
        &= \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + \langle\alpha|\beta\rangle + \langle\beta|\alpha\rangle \\
        &\le \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + 2\langle\alpha|\beta\rangle \\
        &\le \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + 2\left\lVert\alpha\right\rVert\left\lVert\beta\right\rVert \\
        &= (\left\lVert\alpha\right\rVert + \left\lVert\beta\right\rVert)^2
\end{align*}
$$

等于右侧平方，得证。

一如既往地，让我们用 `numpy.array` 探索矢量的内积运算：

In [51]:
# 由于 bra 和 ket 本质是矩阵，它们的积也是一个 1x1 的矩阵，我们应该返回其中的唯一元素
def inner(bra, ket):
    assert(is_bra(bra))
    assert(is_ket(ket))
    return (bra @ ket)[0][0]

a = bra(1j, 2)
b = ket(-1j, 2)
print(inner(a, b))

# 事实上，numpy.inner 可以对一维 numpy.array 进行类似内积的运算，不过它不会自动做复共轭，因此我们不采用这个写法
print(np.inner(np.array([-1j, 2j]), np.array([1, 2])))

(5+0j)
3j


### 矩阵的性质

本节让我们深入探讨矩阵。量子力学中，我们只关心从一个特定空间到其自身的线性变换。此时矩阵的行和列数量相同，我们将其称为 **方阵（Square Matrix）**。后文中不经特别说明，所有提到的矩阵都是方阵。对于一个已知的线性变换 $\hat{T}$，为了得到它的矩阵形式，可以参考下面的等式：

$$
    T_{ij} = \sum_{k=1}^n\delta_{ik}T_{kj} = \sum_{k=1}^n \braket{e_i}{e_k} T_{kj} = \bra{e_i} \sum_{k=1}^n T_{kj}\ket{e_k} = \bracket{e_i}{\hat{T}}{e_j}
$$

整个矩阵则可以通过下面的等式给出：

$$
\begin{equation*}
    \hat{T} = \sum_{i=1}^n\sum_{j=1}^n T_{ij}\ket{e_i}\bra{e_j}
\end{equation*}
$$

下面我们将定义一些矩阵的操作以及特殊矩阵。

- 矩阵 $\hat{T}$ 的 **转置（Transpose）** 定义为其关于主对角线 $T_{ii}$ 的镜像：
    $$
    \begin{equation*}
        (\hat{T}^T)_{ij} = T_{ji}
    \end{equation*}
    $$
    若矩阵的转置和它自身相同，称该矩阵 **对称（Symmetric）**；若其转置满足 $\hat{T}^T = -T$，则称其为 **反对称（Anti-Symmetric）** 的。

- 矩阵的 **复共轭（Complex Conjugate）** 定义为将其每个元素取复共轭的结果：
    $$
    \begin{equation*}
        (T^*)_{ij} = (T_{ij})^*
    \end{equation*}
    $$
    若矩阵的复共轭和它自身相同，称该矩阵为 **实（Real）** 矩阵；若其复共轭满足 $\hat{T}^* = -T$，则称其为 **虚（Imaginary）** 矩阵。

- 矩阵的 **共轭转置（Conjugate Transpose）** 是其共轭的转置，记作 $T^\dagger$。值得一提的是，同一矢量的左矢和右矢互为共轭转置。若矩阵的共轭转置等于其自身，则称其为 **厄米特（Hermitian）** 矩阵。若其共轭转置满足 $T^\dagger = -T$，则称其为 **斜厄米特（Skew Hermitian）** 矩阵。如果矩阵满足 $\hat{T}^\dagger\hat{T} = \hat{T}\hat{T}^\dagger$，我们称其为 **正规矩阵（Normal Matrix）**。显然，厄米特矩阵都是正规矩阵。

- 矩阵的 **协因数阵（Cofactor Matrix）** $\hat{C}$ 为其 **协因数（Cofactor）**（代数余子式） $C_{ij}$ 组成的矩阵。对于任一个元素 $T_{ij}$，其协因数定义为去除 $i$ 行 $j$ 列得到的子矩阵的 **行列式（Determinant）** 乘以 $(-1)^{i+j}$。这里的行列式定义为：
    
    $$
    \begin{equation*}
        \text{det}(\hat{T}) = \sum_{j=1}^nC_{ij}a_{ij}
    \end{equation*}
    $$   
    其中 $i$ 是任取的。协因数和行列式的定义是相互引用的，其基本情形是 $1\times1$ 矩阵，此时它的行列式是其唯一的元素。
    
- **单位矩阵（Identity Matrix）** 定义为除了主对角线上均为 $1$ 外，其它元素均为 $0$ 的矩阵，其满足 $\hat{I}\hat{T} = \hat{T}\hat{I} = \hat{T}$。可以如下表示：

    $$
    \begin{equation*}
        I_{ij} = \delta_{ij}
    \end{equation*}
    $$   
    我们也可以通过任一组基表示出单位矩阵：
    
    $$
    \begin{equation*}
        \hat{I} = \sum_{i=1}^n \ket{e_i}\bra{e_i}
    \end{equation*}
    $$
    验证上式的正确性不难；考察 $\hat{I}\hat{T}$ 的第 $k$ 列，设其为 $|\tau\rangle$：
    
    $$
    \begin{align*}
        \hat{I}\ket{\tau} 
            &= \left(\sum_{i=1}^n\ket{e_i}\bra{e_i}\right)\sum_{i=1}^nt_i|e_i\rangle \\
            &= \sum_{i=1}^n\sum_{j=1}t_j\ket{e_i}\braket{e_i}{e_j} \\
            &= \sum_{i=1}^n\delta_{ij}\sum_{j=1}t_j\ket{e_i} \\
            &= \sum_{i=1}^nt_i\ket{e_i} = \ket{\tau}
    \end{align*}
    $$
    因此 $\hat{I}$ 对 $\hat{T}$ 的每一列都是单位变换，最终整个矩阵也必然是单位变换。
    
- 矩阵的 **逆（Inverse）** 是与其自身相乘得到单位矩阵的唯一矩阵。它可以通过协因数阵得到：
    
    $$
    \begin{equation*}
        \hat{T}^{-1} = \frac{1}{\text{det}(\hat{T})}\hat{C}
    \end{equation*}
    $$
    由于行列式出现在分母，可以预见当 $\text{det}(\hat{T}) = 0$ 时，存在逆的矩阵称为 **可逆矩阵（Invertible Matrix）**，反之称为 **奇异矩阵（Singular Matrix）**。
    
- 若矩阵的逆与其共轭转置相同，则称其为 **幺正矩阵（Unitary Matrix）**。此时有 $\hat{T}^\dagger\hat{T} = I$。注意到幺正矩阵也是正规矩阵。

两个矩阵之积的转置、共轭转置以及逆满足类似的运算规律：

$$
\begin{equation*}
    (\hat{T}\hat{S})^T = \hat{S}^T\hat{T}^T \qquad
    (\hat{T}\hat{S})^\dagger = \hat{S}^\dagger\hat{T}^\dagger \qquad
    (\hat{T}\hat{S})^{-1} = \hat{S}^{-1}\hat{T}^{-1}
\end{equation*}
$$

前两个可以通过其定义证明；第三个式子可以由 $(\hat{S}^{-1}\hat{T}^{-1})(\hat{T}\hat{S}) = \hat{S}^{-1}(\hat{T}^{-1}\hat{T})\hat{S} = \hat{S}^{-1}\hat{S} = I$ 以及逆矩阵的唯一性得到。

现在让我们探索如何从一个基线性变换到另一个基。设原来的基是 $|e_i\rangle$，新的基是 $|e_i'\rangle$。由于在同一个矢量空间中，我们总能找到一系列标量 $T_{ij}$ 使得：

$$
\begin{equation*}
    |e_i\rangle = T_{i1}|e_1'\rangle + \dots + T_{in}|e_n'\rangle
\end{equation*}
$$

此时对任意的 $\ket{\alpha} = \sum_{i=1}^na_i|e_i\rangle$，我们都有：

$$
\begin{equation*}
    a_i\ket{e_i} = a_i\sum_{j=1}^nT_{ij}\ket{e_j'}
\end{equation*}
$$

因此有：

$$
\begin{equation*}
    \ket{\alpha'} = \hat{T}\ket{\alpha}
\end{equation*}
$$

对于作用在原来的基上的线性变换 $\hat{\mathcal{O}}$，我们不难得到其在新的基上的形式：

$$
    \hat{\mathcal{O}'} = \hat{T}\hat{\mathcal{O}}\hat{T}^{-1}
$$

这是因为变换一个 $|e_i'\rangle$ 上的矢量等价于将其转换成 $\ket{e_i}$ 上的矢量，进行 $\hat{T}$ 变换，然后重新转换为 $\ket{e_i'}$ 上的矢量。这样的两个矩阵 $\hat{\mathcal{O}}$ 和 $\hat{\mathcal{O}'}$ 被称为 **相似（Similar）** 的（因为它们本质做的是完全相同的线性变换，只不过作用于不同的基）。

### 特征值与特征矢量

对于任意的矩阵 $\hat{T}$，我们总能找到一些特殊的矢量 $\ket{\alpha}$ 使得下式成立：

$$
\begin{equation*}
    \hat{T}\ket{\alpha} = \lambda\ket{\alpha}
\end{equation*}
$$

这样的矢量被称为 $\hat{T}$ 的 **特征矢量（Eigenvector）**，而对应的 $\lambda$ 被称为 **特征值（Eigenvalue）**。为了求出矩阵的特征矢量，可以列出下式：

$$
\begin{equation*}
    (\hat{T} - \lambda\hat{I})\ket{\alpha} = \ket{0} \implies \text{det}(T - \lambda\hat{I}) = 0
\end{equation*}
$$

之后经过整理我们可以得到一个 $n$ 次的方程：

$$
\begin{equation*}
    c_n\lambda^n + \dots + c_0 = 0
\end{equation*}
$$

根据代数基本定理，上面的方程一定能找到 $n$ 的复数解 $\lambda_1, \dots, \lambda_n$，称为矩阵 $\hat{T}$ 的 **波谱（Spectrum）**。一些情况下，波谱中会出现重复的特征值，此时我们称其为 **退化（Degenerate）** 的波谱。当一个特征值重复 $d$ 次时，我们称其拥有 **重数（Multiplicity）** $d$。不难认识到 $\sum_{i=1}^nd_i = \text{dim}(V)$。

当一个波谱可以张成整个矢量空间时，我们可以尝试将其对应特征矢量作为矢量空间的基。观察到：

$$
\begin{equation*}
    \hat{T}\sum_{i=1}^n a_i\ket{e_i} = \sum_{i=1}^n \lambda_i\ket{e_i} = \begin{pmatrix}
        \lambda_1 & \dots & 0 \\
        \vdots & \ddots & 0 \\
        0 & \dots & \lambda_n
    \end{pmatrix}
\end{equation*}
$$

我们将其成为矩阵 $\hat{T}$ 的 **对角化形式（Diagonal Form）**，这样仅在主对角线上有非零元素的矩阵称为 **对角矩阵（Diagonal Matrix）**。特征矢量能张成矢量空间的矩阵被称为 **可对角化（Diagonalizable）** 的，因为它拥有对角化形式。对角化的矩阵因为其简单性，我们在计算中会青睐于使用这种形式。

定义矩阵的 **迹（Trace）** 为其主对角线上元素之和，即：

$$
\begin{equation*}
    \text{tr}(\hat{T} = \sum_{i=0}^nT_{ii}
\end{equation*}
$$

矩阵迹有下面的一些重要特征：

- 它是线性的，因此 $\text{tr}(a\hat{T} + \hat{U}) = a\text{tr}(\hat{T}) + \text{tr}(\hat{U})$。
- 它有循环性，比如 $\text{tr}(\hat{T}\hat{U}\hat{S}) = \text{tr}(\hat{U}\hat{S}\hat{T}) = \text{tr}(\hat{S}\hat{T}\hat{U})$。
- 它与基的选取无关。对于任选的两个基 $\ket{u_i}$ 和 $\ket{v_i}$ 都有：
    
    $$
    \begin{equation*}
        \text{tr}(\hat{T}) = \sum_{i=1}^n \bracket{u_i}{\hat{T}}{u_i} = \sum_{i=1}^n \bracket{v_i}{\hat{T}}{v_i}
    \end{equation*}
    $$

- 它等于矩阵所有特征值受重数的加权和：

    $$
    \begin{equation*}
        \text{tr}(\hat{T}) = \sum_{i=1}^n d_i\lambda_i
    \end{equation*}
    $$
    
这里值得再提一下行列式。我们此前为了定义逆矩阵，用协因数阵辅助定义了行列式。事实上通过特征值，我们可以得到更加优雅的定义。矩阵的行列式为其特征值受重数的加权积，即：
    $$
    \begin{equation*}
        \text{tr}(\hat{T}) = \prod_{i=1}^n \lambda_i^{d_i}
    \end{equation*}
    $$
    
行列式在量子力学中的用处不多，因此在这里就不深入探索了

### 正规矩阵

本节介绍在量子力学与量子计算中有特别地位的厄米特矩阵和幺正矩阵。首先让我们了解正规矩阵的性质。回忆正规矩阵是满足 $\hat{T}\hat{T}^\dagger = \hat{T}^\dagger\hat{T}$ 的矩阵。

- 正规矩阵和它的共轭转置作用于任意矢量，其结果的范数相同：
    $$
    \begin{equation*}
        \braket{\alpha}{(\hat{T}^\dagger\hat{T} - \hat{T}\hat{T}^\dagger)\alpha} = 0 \implies
        \braket{\alpha}{\hat{T}^\dagger\hat{T}\alpha} = \braket{\alpha}{\hat{T}\hat{T}^\dagger\alpha} \implies
        \left\lVert\hat{T}\alpha\right\rVert^2 = \left\lVert\hat{T}^\dagger\alpha\right\rVert^2
    \end{equation*}
    $$
- 正规矩阵和它的共轭转置拥有相同的特征矢量。首先不难验证 $\hat{T} - \lambda\hat{I}$ 也是正规矩阵，因此：
    $$
    \begin{equation*}
        0 = \left\lVert(\hat{T} - \lambda\hat{I})\alpha\right\rVert = \left\lVert(\hat{T} - \lambda\hat{I})^\dagger\alpha\right\rVert = \left\lVert(\hat{T}^\dagger - \lambda^*\hat{I})\alpha\right\rVert
    \end{equation*}
    $$ 
- **波谱定理（Spectral Theorem）**：对于矢量空间 $V$ 上的线性变换 $\hat{T}$ 是正规矩阵，当且仅当 $\hat{T}$ 的特征矢量是 $V$ 的一个标准正交基。

#### 厄米特矩阵

回忆厄米特矩阵是共轭转置等于其自身的矩阵。它广泛出现于量子力学中，并满足一个重要的等式关系：

$$
\begin{equation*}
    \braket{\hat{T}\alpha}{\beta} = \braket{\alpha}{\hat{T}\beta}
\end{equation*}
$$

注意上面的记号 $\bra{\hat{T}\alpha}$ 指的是 $\hat{T}\ket{\alpha}$ 对应的左矢。下面是一个简单的证明：

$$
\begin{equation*}
    \braket{\hat{T}\alpha}{\beta} = (T^\dagger\ket{\alpha})^\dagger\ket{\beta} = \ket{\alpha}^\dagger (T\ket{\beta}) = \braket{\alpha}{T\beta}
\end{equation*}
$$

厄米特矩阵还有一些重要的性质：

- 其特征值一定是实数。设其特征矢量为 $\ket{\alpha}$，则：
    $$
    \begin{equation*}
        \braket{\alpha}{\hat{T}\alpha} = \braket{\alpha}{\lambda\alpha} = \lambda\sbraket{\alpha}
    \end{equation*}
    $$
    同时根据此前推得的等式：  
    $$
    \begin{equation*}
        \braket{\alpha}{\hat{T}\alpha} = \braket{\hat{T}\alpha}{\alpha} = \braket{\lambda\alpha}{\alpha} = \lambda^*\sbraket{\alpha}
    \end{equation*}
    $$
    因此有 $\lambda = \lambda^*$，故特征值为实数。反过来说，特征值为实数的矩阵一定是厄米特矩阵。
    
- 其不同特征值对应的特征矢量相互正交。设特征矢量 $\ket{\alpha}$ 和 $\ket{\beta}$ 分别对应不同的特征值 $\lambda$ 与 $\mu$，则：
    $$
    \begin{equation*}
        \braket{\alpha}{\hat{T}\beta} = \braket{\alpha}{\mu\beta} = \mu\braket{\alpha}{\beta}
    \end{equation*}
    $$
    同时：  
    $$
    \begin{equation*}
        \braket{\alpha}{\hat{T}\beta} = \braket{\hat{T}\alpha}{\beta} = \braket{\lambda\alpha}{\beta} = \lambda^*\braket{\alpha}{\beta}
    \end{equation*}
    $$
    由于 $\lambda$ 一定是实数，这里如果 $\braket{\alpha}{\beta} \ne 0$ 我们就得到 $\mu = \lambda^* = \lambda$，与题设矛盾。因此一定有 $\ket{\alpha}$ 和 $\ket{\beta}$ 正交。
    
- 其是可对角化的，因为它是正规矩阵。

#### 幺正矩阵

回忆幺正矩阵为逆等于共轭转置的矩阵，即 $\hat{T}\hat{T}^\dagger = I$。它的部分特点如下：

- 其行列式绝对值为 $1$。考虑行列式的运算特性：
    $$
    \begin{equation*}
        |\text{det}(\hat{T})|^2 = \text{det}(\hat{T})[\text{det}(\hat{T})]^* = \text{det}(\hat{T}\hat{T}^\dagger) = \text{det}(I) = 1 
    \end{equation*}
    $$
    因此幺正矩阵的行列式，其绝对值为 $1$。特别地，如果它同时是哈密顿矩阵，其行列式只能是 $\pm 1$。

- 幺正矩阵同时作用于内积的左右矢时不影响结果，即：
    $$
    \begin{equation*}
        \braket{\hat{U}\alpha}{\hat{U}\beta} = (\hat{U}\ket{\alpha})^\dagger(\hat{U}\ket{\beta}) = \bra{\alpha}\hat{U}^\dagger\hat{U}\ket{\beta} = \braket{\alpha}{\beta}
    \end{equation*}
    $$
- 其特征值的绝对值为 $1$。证明如下：
    $$
    \begin{equation*}
        |\lambda|^2\sbraket{\alpha} = \sbraket{\lambda\alpha} = \sbraket{\hat{U}\alpha} = \sbraket{\alpha}
    \end{equation*}
    $$
- 其是可对角化的，因为它是一个正规矩阵。

若 $\hat{U}$ 同为厄米特矩阵且幺正矩阵，我们得到一个有趣的性质：

$$
\begin{equation*}
    \hat{U}^2 = \hat{U}^\dagger\hat{U} = \hat{U}^{-1}\hat{U} = \hat{T}
\end{equation*}
$$

此时我们可以尝试将矩阵运算拓展到一些常用函数上，得到相当特别的结果：

- 指数函数 $e^x$。考虑 $e^{i\theta\hat{U}}$，使用 $x=0$ 处的泰勒展开：
    $$
    \begin{align*}
        e^{i\theta\hat{U}}
            &= \hat{I} + i\theta\hat{U} + \frac{1}{2!}(i\theta)^2\hat{U}^2 + \frac{1}{3!}(i\theta)^3\hat{U}^3 + \dots \\
            &= \hat{I}\left( 1 - \frac{1}{2!}\theta^2 + \frac{1}{4!}\theta^4 - \dots \right) + i\hat{U}\left( \theta - \frac{1}{3!}\theta^3 + \frac{1}{5!}\theta^5 - \dots \right) \\
            &= \hat{I}\cos\theta + i\hat{U}\sin\theta
    \end{align*}
    $$

### 张量积

矩阵的 **张量积（Tensor Product）**，或 **克罗内克积（Kronecker Product）** 定义如下：

$$
\begin{equation*}
    \hat{A}\otimes\hat{B} = \begin{pmatrix}
        A_{11}\hat{B} & \dots & A_{nn}\hat{B} \\
        \vdots & \ddots & \vdots \\
        A_{n1}\hat{B} & \dots & A_{nn}\hat{B}
    \end{pmatrix} = \begin{pmatrix}
        A_{11}B_{11} & \dots & A_{11}B_{1n} & \dots & A_{1n}B_{11} & \dots & A_{1n}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{11}B_{n1} & \dots & A_{11}B_{nn} & \dots & A_{1n}B_{n1} & \dots & A_{1n}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{n1}B_{11} & \dots & A_{n1}B_{1n} & \dots & A_{nn}B_{11} & \dots & A_{nn}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{n1}B_{n1} & \dots & A_{n1}B_{nn} & \dots & A_{nn}B_{n1} & \dots & A_{nn}B_{nn} \\
    \end{pmatrix}
\end{equation*}
$$

特别地，若 $A$ 是一个右矢 $\ket{\alpha}$，$B$ 是一个左矢 $\bra{\beta}$，上面等同于：

$$
\begin{equation*}
    \ket{\alpha}\otimes\bra{\beta} = \ket{\alpha}\bra{\beta}
\end{equation*}
$$

当两个矩阵都是右矢时，我们会在左右矢符号中将张量积符号省略，写作：

$$
\begin{equation*}
    \ket{\psi\phi} = \ket{\psi}\otimes\ket{\phi}
\end{equation*}
$$

注意这里的简写并不如最开始提到的那种直观；$\ket{\psi + \phi} \equiv \ket{\psi} + \ket{\phi}$、$\ket{c\psi} = c\ket{\psi}$，然而 $\ket{\psi\phi} = \ket{\psi}\otimes\ket{\phi}$，需要适应一下（一些人会直接将张量积完全省略，变为 $\ket{\psi}\ket{\phi}$，但这和矩阵积的表示方法发生冲突，在一些情况下会造成歧义，因此我不采用这个写法）。

虽然张量积看起来将两个矩阵的内容完全混杂起来了，但在实际使用时两者可以很清晰地区分。考虑 $\hat{M}_1 = \hat{A}_1\otimes\hat{B}_1$ 而 $\hat{M}_2 = \hat{A}_2\otimes\hat{B}_2$，此时它们的矩阵积为：

$$
\begin{equation*}
    \hat{M}_1\hat{M}_2 = (\hat{A}_1\otimes\hat{B}_1)(\hat{A}_2\otimes\hat{B}_2) = (\hat{A}_1\hat{A}_2)\otimes(\hat{B}_1\hat{B}_2)
\end{equation*}
$$

上面这个公式非常重要，我们会在后面量子电路中频繁使用。特别地，对于左矢和右矢的运算，我们有：

$$
\begin{equation*}
    \braket{\psi_1\phi_1}{\psi_2\phi_2} = \braket{\psi_1}{\psi_2}\braket{\phi_1}{\phi_2}
\end{equation*}
$$

且对于矩阵 $\hat{T}_1$ 和 $\hat{T}_2$，我们可以将其作用于 $\ket{\psi} = \ket{\psi_1\psi_2}$ 等价写为：

$$
\begin{equation*}
    (\hat{T}_1\otimes\hat{T}_2)\ket{\psi} = (\hat{T}_1\otimes\hat{T}_2)(\ket{\psi_1\psi_2}) = (\hat{T}_1\ket{\psi_1})\otimes(\hat{T}_2\ket{\psi_2}) = \ket{\hat{T}_1\psi_1\,\hat{T}_2\psi_2}
\end{equation*}
$$

张量积会保留矩阵的一些好性质。比如：

- 若 $\hat{A}$、$\hat{B}$ 均为厄米特矩阵，则 $\hat{A}\otimes\hat{B}$ 也是厄米特矩阵。
- 若 $\hat{A}$、$\hat{B}$ 均为幺正矩阵，则 $\hat{A}\otimes\hat{B}$ 也是幺正矩阵。

## 量子力学简介

### 波函数与期望

经典力学中，我们可以根据时间确定物体的准确位置，即通过函数 $x(t)$ 描述物体（本篇笔记中我们只考虑一维的空间）。但是实验证明，在微观情况下，**量子（Quantum）**，即物体的最小单元，其在某一刻的位置是由分布 $\psi(x)$ 决定的。对于任意一点 $x_0$ 上其出现的概率密度为 $|\psi(x_0)|^2$。由于概率加和应为 $1$，我们有：

$$
\begin{equation*}
    \int_{-\infty}^\infty |\psi(x_0)|^2\,dx = 1
\end{equation*}
$$

特别地，粒子在 $a \le x \le b$ 上出现的概率为：

$$
\begin{equation*}
    \text{Pr}(a \le x \le b) = \int_a^b |\psi(x_0)|^2\,dx
\end{equation*}
$$

从统计角度出发，粒子所在位置的期望值为：

$$
\begin{equation*}
    \exp{x} = \int_{-\infty}^\infty x|\psi(x)|^2\,dx
\end{equation*}
$$

但如果我们需要的是例子的动量，以目前已有的公式似乎推断不出（$p$ 是一个和 $x$ 相关的量）。为了能够继续，让我们首先对这个神秘的分布进行建模。首先，我们假定波函数可以被通过某个平面波描述：

$$
    \Psi(x, t) = Ae^{i(kx - \omega t)}
$$

其中 $k$ 时 **波数（Wave Number）**、$\omega$ 是 **角频率（Angular Frequency）**。此时它的偏导数为：

$$
\begin{equation*}
    \frac{\partial \Psi}{\partial t} = -i\omega\Psi(x, t) \qquad \frac{\partial^2 \Psi}{\partial x^2} = -k^2\Psi(x, t)
\end{equation*}
$$

根据波数的定义 $k = \frac{2\pi}{\lambda}$ 以及德布罗意公式的变形 $p = \frac{2\pi\hbar}{\lambda}$，我们有：

$$
\begin{equation*}
    k = \frac{p}{\hbar}
\end{equation*}
$$

哈密顿量 $H = \frac{p^2}{2m} + V(x)$，考察 $H\Psi(x, t)$ 有：

$$
\begin{align*}
    H\Psi(x, t)
        &= \frac{p^2}{2m}\Psi(x, t) + V(x)\Psi(x, t) \\
        &= -\frac{\hbar^2}{2m}\frac{\partial^2 \Psi}{\partial x^2} + V(x)\Psi(x, t)
\end{align*}
$$

对于粒子同时有 $H = \hbar\omega$，因此上面等式左侧变形为 $i\hbar\frac{\partial \Psi}{\partial t}$。最后我们得到：

$$
\begin{equation*}
    i\hbar\frac{\partial \Psi}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2\Psi}{\partial x^2} + V(x)\Psi
\end{equation*}
$$

这便是鼎鼎大名的 **薛定谔方程（Schrodinger's Equation）**。其用于描述量子力学中所有的波函数。它另一种常见的形式没有将哈密顿量拆开：

$$
\begin{equation*}
    H\Psi = i\hbar\frac{\partial\Psi}{\partial t}
\end{equation*}
$$

现在我们可以尝试计算动量的期望了。回忆 $p = m\frac{dx}{dt}$，因此：

$$
\begin{equation*}
    \frac{d\exp{x}}{dt} = \frac{d}{dt} \int x|\Psi|^2\,dx = \int x \frac{\partial}{\partial t}\left(\Psi^*\Psi\right)\,dx = \int x\left(\frac{\partial\Psi^*}{\partial t} + \frac{\partial \Psi}{\partial t}\right)\,dx
\end{equation*}
$$

根据薛定谔方程，我们有：

$$
\begin{equation*}
    \frac{\partial\Psi}{\partial t} = \frac{i\hbar}{2m}\frac{\partial^2\Psi}{\partial x^2} - \frac{i}{\hbar}V\Psi \qquad
    \frac{\partial\Psi^*}{\partial t} = -\frac{i\hbar}{2m}\frac{\partial^2\Psi^*}{\partial x^2} + \frac{i}{\hbar}V\Psi^*
\end{equation*}
$$

代入原来的式子：

$$
\begin{equation*}
    \frac{d\exp{x}}{dt} = -\frac{i\hbar}{2m}\int\left(\Psi^*\frac{\partial\Psi}{\partial x} - \frac{\partial\Psi^*}{\partial x}\Psi\right)\,dx = -\frac{i\hbar}{m}\int\Psi^*\frac{\partial\Psi}{\partial x}\,dx
\end{equation*}
$$

因此：

$$
\begin{equation*}
    \exp{p} = -i\hbar\int\Psi^*\frac{\partial\Psi}{\partial x}\,dx
\end{equation*}
$$

和位置的期望值对比，我们会发现它们都有一个特征，即：

$$
\begin{equation*}
    \exp{Q} = \int\Psi^*[\hat{Q}]\Psi\,dx
\end{equation*}
$$

对于位置来说，我们有 $\hat{x} = x$，而对于动量，则有：

$$
\begin{equation*}
    \hat{p} = -i\hbar\frac{\partial}{\partial x}
\end{equation*}
$$

我们将这些在量子力学中表示物理量的期望特征的对象成为 **算符（Operator）**，用物理量上加尖角符号的形式表示。读者可能认为这和矩阵的表示方式重复了，但很快我们会提到两者之间的联系。

### 希尔伯特空间

本节中，我们会揭示量子力学和线性代数的联系。对于任意波函数 $\Psi$，其必须是 **归一化（Normalized）** 的，即：

$$
\begin{equation*}
    \int_{-\infty}^\infty |\Psi|^2\,dx = 1
\end{equation*}
$$

如果我们将情况一般化，其相当于要求一个函数在特定区间中 **平方可积（Square-Integrable）**：

$$
\begin{equation*}
    \int_a^b |\Psi|^2\,dx < \infty
\end{equation*}
$$

我们将所有满足这样要求的函数组成的空间称为 **希尔伯特空间（Hilbert Space）**，数学上记作 $L^2(a, b)$。下面我们将尝试把它构建为一个矢量空间。设 $f$、$g$ 为希尔伯特空间中任取的两个函数：

- 定义矢量加法为 $(f + g)(x) = f(x) + g(x)$。
- 定义标量乘法为 $(cf)(x) = cf(x)$。

今后我们将统一把波函数写成矢量的形式（除了在积分中）。此外，希尔伯特空间也是一个内积空间，定义内积 $\braket{f}{g}$ 为：

$$
\begin{equation*}
    \braket{f}{g} = \int_a^b f^*(x)g(x)\,dx
\end{equation*}
$$

不难验证这个定义满足内积的要求：

$$
\begin{align*}
    \braket{g}{f} &= \int_a^b g^*(x)f(x)\,dx = \int_a^b (f^*(x)g(x))^*\,dx = \braket{f}{g}^* \\
    \braket{f}{f} &= \int_a^b f^*(x)f(x)\,dx = \int_a^b |f(x)|^2\,dx \ge 0\\
    \braket{af}{g + h} &= \int_a^b (af)^*(x)(g(x) + h(x))\,dx = \int_a^b a^*f^*(x)g(x) + \int_a^b a^*f^*(x)h(x) = a^*\braket{f}{g} + a^*\braket{f}{h}
\end{align*}
$$

现在，我们可以正式让波函数继承所有矢量的语言；比如两个正交的波函数定义为 $\braket{f}{g} = 0$，一集标准正交的函数 $f_n$ 需要满足 $\braket{f_i}{f_j} = \delta_{ij}$ 等等。特别地，如果一集函数能张成整个希尔伯特空间，我们称它们是 **完备（Complete）** 的，此时对于任意函数 $f(x)$ 都有：

$$
\begin{equation*}
    f(x) = \sum_{n=1}^\infty c_nf_n(x)
\end{equation*}
$$

为了找到这些系数 $c_n$，我们依然可以依照此前的公式（关于这个，有一个更加详细且复杂的证明过程，这里不展开了）：

$$
\begin{equation*}
    c_n = \braket{f_n}{f} = \int_a^b f_n^*(x)f(x)\,dx
\end{equation*}
$$

现在让我们考察物理量在线性代数中的含义。量子力学常将它们称为 **可观测量（Observables）**，其总是对应一个算符。上一节中我们尝试总结了一个规律，即可观测量的期望等于下面的积分：

$$
\begin{equation*}
    \exp{Q} = \int \Psi^*\hat{Q}\Psi\,dx = \braket{\Psi}{\hat{Q}\Psi}
\end{equation*}
$$

由于可观测量一定是实数，我们有 $\exp{Q} = \exp{Q}^*$，此时：

$$
\begin{equation*}
    \braket{\Psi}{\hat{Q}\Psi} = \exp{Q} = \exp{Q}^* = \braket{\hat{Q}\Psi}{\Psi}
\end{equation*}
$$

因此可观测量对应的算符一定是厄米特矩阵。这也是为什么我此前让算符和矩阵采用同样的标记。算符自此会继承所有矩阵的术语和性质，我会用相同的方式处理两者（除了在显示写出矩阵的元素时）。

最后，我们还应将可观测量纳入到线性代数的语系中。注意到通常来讲，同样的状态 $\ket{\Psi}$ 下，每次测量的结果一般是不同的。在某些特殊状态，称为 **确定状态（Determinate State）** 下，或许我们能保证测量结果的稳定性。这实际上需要可观测量的标准差为零，即：

$$
\begin{equation*}
    \sigma^2 = \exp{(Q - \exp{Q})^2} = \braket{\Psi}{(\hat{Q} - \exp{Q})^2\Psi} = \braket{(\hat{Q} - \exp{Q})\Psi}{(\hat{Q} - \exp{Q})\Psi} = 0
\end{equation*}
$$

因此：

$$
\begin{equation*}
    \hat{Q}\ket{\Psi} = \exp{Q}\ket{\Psi}
\end{equation*}
$$

所以可观测量的期望值是其对应算符的特征值，而它的确定状态是这些特征值对应的特征矢量。

### 量子力学的统计解释

有了之前的所有铺垫，我们可以用正规的语言描述量子力学的统计意义了：在 $\Psi(x, t)$ 状态下测量粒子的可观测量 $Q(x, p)$ 时，我们一定会得到厄米特算符 $\hat{Q}$ 的一个特征值。对于离散波谱（即可数个特征值），每个特征值 $q_n$ 及特征矢量 $\ket{f_n}$ 出现的概率为 $|\braket{f_n}{\Psi}|^2$。这个过程称为 **波函数坍缩（Wave Function Collapse）**。

考虑两个可观测量 $A$ 与 $B$，它们的标准差满足：

$$
\begin{align*}
    \sigma_A^2 = \braket{(\hat{A} - \exp{A})\Psi}{(\hat{A} - \exp{A})\Psi} \\
    \sigma_B^2 = \braket{(\hat{B} - \exp{B})\Psi}{(\hat{B} - \exp{B})\Psi}
\end{align*}
$$

回忆施瓦兹不等式 $\sbraket{f}\sbraket{g} \ge |\braket{f}{g}|^2$，因此：

$$
\begin{equation*}
    \sigma_A^2\sigma_B^2
        \ge \left|\braket{(\hat{A} - \exp{A})\Psi}{(\hat{B} - \exp{B})\Psi}\right|^2 
\end{equation*}
$$

令 $z = \braket{(\hat{A} - \exp{A})\Psi}{(\hat{B} - \exp{B})\Psi)}$，复数拥有下面的性质：

$$
\begin{equation*}
    |z|^2 = [\text{Re}(z)]^2 + [\text{Im}(z)]^2 \ge [\text{Im}(z)]^2 = \left[\frac{1}{2i}(z - z^*)\right]^2
\end{equation*}
$$

同时：

$$
\begin{align*}
    z
        &= \braket{\Psi}{(\hat{A} - \exp{A})(\hat{B} - \exp{B})\Psi} \\
        &= \braket{\Psi}{\hat{A}\hat{B}\Psi} - \exp{A}\braket{\Psi}{\hat{B}\Psi} - \exp{B}\braket{\Psi}{\hat{A}\Psi} + \exp{A}\exp{B}\sbraket{\Psi} \\
        &= \exp{\hat{A}\hat{B}} - \exp{A}\exp{B} - \exp{B}\exp{A} + \exp{A}\exp{B} \\
        &= \exp{\hat{A}\hat{B}} - \exp{A}\exp{B}
\end{align*}
$$

总结下来：

$$
\begin{align*}
    \sigma_A^2\sigma_B^2 
         &\ge \left[\frac{1}{2i}\left(\exp{\hat{A}\hat{B}} - \exp{A}\exp{B} - \exp{\hat{B}\hat{A}} + \exp{B}\exp{A}\right)\right]^2 \\
         &= \left[\frac{1}{2i}\exp{\hat{A}\hat{B} - \hat{B}\hat{A}}\right]^2
\end{align*}
$$

我们将其中的 $\hat{A}\hat{B} - \hat{B}\hat{A}$ 定义为算符 $\hat{A}$、$\hat{B}$ 的 **交换子（Commutator）**，记为：

$$
\begin{equation*}
    [\hat{A}, \hat{B}] = \hat{A}\hat{B} - \hat{B}\hat{A}
\end{equation*}
$$

最终得到：

$$
\begin{equation*}
    \sigma_A^2\sigma_B^2 \ge \left(\frac{1}{2i}[\hat{A}, \hat{B}]\right)^2
\end{equation*}
$$

这个结论告诉我们的是，某个对两个可观测量均为确定的状态存在当且仅当它们的算符 **互易（Commute）**，即 $[\hat{A}, \hat{B}] = 0$。事实上，多数的算符之间都不可互易，比如位置和动量：

$$
\begin{align*}
    [\hat{x}, \hat{p}] f(x) 
        &= x\cdot i\hbar\frac{\partial}{\partial x} f(x) - i\hbar\frac{\partial}{\partial x} xf(x) \\
        &= i\hbar xf'(x) - i\hbar f(x) - i\hbar xf'(x) \\
        &= i\hbar f(x)
\end{align*}
$$

这就得到了 **标准对易关系（Canonical Commutation Relation）**：

$$
\begin{equation*}
    [\hat{x}, \hat{p}] = i\hbar
\end{equation*}
$$

代入此前标准差的不等式后有：

$$
\begin{equation*}
    \sigma_x\sigma_p \ge \frac{\hbar}{2}
\end{equation*}
$$

这便是著名的 **测不准原理（Uncertainty Principle）**。对于这类对易子不为零的可观测量，我们称其相互 **不兼容（Incompatible）**。它们不存在相同的特征矢量。注意到一个正规矩阵，其自身和转置共轭总是互易的。

### 自旋

对于粒子来说，其角动量由 **轨道（Orbital）** 角动量和 **自旋（Spin）** 构成；前者是粒子关于某个中心旋转导致的，其通过 $\mathbf{L} = \mathbf{r}\times\mathbf{p}$ 计算，后者则通过 $\mathbf{S} = I\boldsymbol{\omega}$ 得到。关于自旋，我们常通过其 $x$、$y$、$z$ 方向的分量描述，即 $S_x$、$S_y$、$S_z$。它们满足下面的对易关系：

$$
\begin{equation*}
    [\hat{S}_x, \hat{S}_y] = i\hbar \hat{S}_z \quad
    [\hat{S}_y, S_z] = i\hbar \hat{S}_x \quad
    [\hat{S}_z, S_x] = i\hbar \hat{S}_y
\end{equation*}
$$

这些等式可以暂时看成公理。可以看到三个方向上的自旋之间相互不兼容；有趣的是，它们中的每一个都和自旋的平方互易：

$$
\begin{align*}
    [\hat{S}^2, \hat{S}_x]
        &= [\hat{S}_x^2, \hat{S}_x] + [\hat{S}_y^2, \hat{S}_x] + [\hat{S}_z^2, \hat{S}_x] \\
        &= 0 + \hat{S}_y[\hat{S}_y, \hat{S}_x] + [\hat{S}_y, \hat{S}_x]\hat{S}_y + S_z[\hat{S}_z, \hat{S}_x] + [\hat{S}_z, \hat{S}_x]\hat{S}_z \\
        &= \hat{S}_y(-i\hbar \hat{S}_z) + (-i\hbar \hat{S}_z)\hat{S}_y + \hat{S}_z(i\hbar \hat{S}_y) + (i\hbar \hat{S}_y)\hat{S}_z \\
        &= 0
\end{align*}
$$

其等价的另一种形式为：

$$
\begin{equation*}
    [S^2, \mathbf{S}] = 0
\end{equation*}
$$

假设某个 $S_z$ 和 $S^2$ 的共同特征矢量为 $f$，则有：

$$
\begin{equation*}
    S^2f = \lambda f \qquad S_zf = \mu f
\end{equation*}
$$

为了展现特征值的分布，让我们引入 **阶梯算符（Ladder Operator）** $S_\pm$，其定义为：

$$
\begin{equation*}
    S_\pm = S_x \pm iS_y
\end{equation*}
$$

不难验证 $[S_z, S_\pm] = \pm\hbar S_\pm$，而 $[S^2, S_\pm] = 0$、注意到 $L_\pm f$ 同时也是 $S^2$ 和 $S_z$ 的特征矢量，其对应的特征值和 $f$ 相同：

$$
\begin{equation*}
    S^2(S_\pm f) = S_\pm(S^2 f) = S_\pm(\lambda f) = \lambda (S_\pm f)
\end{equation*}
$$

对于 $S_z$ 则有：

$$
\begin{equation*}
    S_z(S_\pm f) = [S_z, S_\pm]f + S_\pm S_zf = \pm\hbar S_\pm f + S_\pm(\mu f) = (\mu \pm \hbar)(S_\pm f)
\end{equation*}
$$

可以看到，$S_\pm$ 作用于函数 $f$ 上时会将其特征值提升或降低 $\hbar$，这也是阶梯算符名称的由来。特别地，$S_+$ 称为 **上升算符（Raising Operator）**，而 $S_-$ 称为 **下降算符（Lowering Operator）**。这样的上升和下降并不是没有限制的，其最小不能低于零，最高不能高于总量 $S$。假设这个最高点的特征矢量为 $f_t$，其对应的特征值为 $\hbar s$，此时：

$$
\begin{equation*}
    S_zf_t = \hbar sf_t \quad S^2f_t = \lambda f_t \quad S_+f_t = 0
\end{equation*}
$$

观察 $S_\pm S_\mp$：

$$
\begin{align*}
    S_\pm S_\mp
        &= (S_x \pm iS_y)(S_x \mp iS_y) \\
        &= S_x^2 + S_y^2 \mp i[S_x, S_y] \\
        &= S^2 - S_z^2 \pm \hbar S_z
\end{align*}
$$

将其中的 $S^2$ 用其它部分表示，则有：

$$
\begin{align*}
    S^2f_t
        &= (S_-S_+ + S_z^2 + \hbar S_z)f_t \\
        &= S_-(S_+f_t) + S_z^2f_t + \hbar S_zf_t \\
        &= \hbar^2s(s + 1)f_t
\end{align*}
$$

因此 $\lambda$ 和 $s$ 的关系为：

$$
\begin{equation*}
    \lambda = \hbar^2s(s + 1)
\end{equation*}
$$

类似地，如果设最低点的特征矢量为 $f_b$，其对应的特征值为 $\hbar \bar{s}$，我们可以得到：

$$
\begin{equation*}
    \lambda = \hbar^2\bar{s}(\bar{s} + 1)
\end{equation*}
$$

由 $s(s + 1) = \bar{s}(\bar{s} - 1)$，唯一合理的情形是 $\bar{s} = -s$。因此，粒子自旋的平方以及其 $z$ 方向上自旋分量满足下面的式子：

$$
\begin{equation*}
    S^2f_s^m = \hbar s(s + 1)f_s^m \qquad
    S_zf_s^m = \hbar mf_s^m
\end{equation*}
$$

其中 $s$ 可以是任意的整数或半整数，如 $0$、$3/2$ 等；而 $m$ 是从 $-s$ 到 $s$ 的所有整数（$s$ 是整数时）或所有半整数（$s$ 是半整数时）。一个更加亲切的写法是：

$$
\begin{equation*}
    S^2\ket{s\ m} = \hbar s(s + 1)\ket{s\ m} \qquad 
    S_z\ket{s\ m} = \hbar m\ket{s\ m}
\end{equation*}
$$

当 $s=0$ 时，整个系统只有一个特征矢量，它没有什么特别的。一个重要的自旋数是 $1/2$，组成所有物质的常规粒子，如质子、中子、电子都是 $1/2$ 自旋的。此时有两个特征矢量，分别是 $\ket{\frac{1}{2}\ \frac{1}{2}}$ 和 $\ket{\frac{1}{2}\ -\frac{1}{2}}$。我们分别将其称为 **上自旋（Spin Up）** 和 **下自旋（Spin Down）**，在量子力学中用 $\ket{\uparrow}$ 和 $\ket{\downarrow}$ 表示。对于 $1/2$ 自旋粒子的任何状态，都可以通过上下自旋的线性组合表示：

$$
\begin{equation*}
    \ket{\chi} = \alpha\ket{\uparrow} + \beta\ket{\downarrow}
\end{equation*}
$$

其中 $|\alpha|^2 + |\beta|^2 = 1$。因此，$1/2$ 自旋的粒子处于一个二维的希尔伯特空间中。下面让我们尝试得到 $S^2$ 和 $S_z$ 的矩阵形式：

$$
\begin{align*}
    \left.
    \begin{split}
        \hat{S}^2\ket{\uparrow} = \frac{3}{4}\hbar^2\ket{\uparrow} \\
        \hat{S}^2\ket{\downarrow} = \frac{3}{4}\hbar^2\ket{\downarrow}
    \end{split}\right\}
    &\implies \hat{S}^2 = \frac{3}{4}\hbar^2\hat{I} \\
    \left.
    \begin{split}
        \hat{S}_z\ket{\uparrow} = \frac{\hbar}{2}\ket{\uparrow} \\
        \hat{S}_z\ket{\downarrow} = -\frac{\hbar}{2}\ket{\downarrow}
    \end{split}\right\}
    &\implies \hat{S}_z = \frac{\hbar}{2}\begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
\end{align*}
$$

经过一些计算后，我们也可以得到 $\hat{S}_x$ 和 $\hat{S}_y$ 的矩阵形式：

$$
\begin{equation*}
    \hat{S}_x = \frac{\hbar}{2}\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \quad
    \hat{S}_y = \frac{\hbar}{2}\begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
\end{equation*}
$$

显然，$\hat{S}_z$ 的特征矢量是 $\ket{\uparrow}$ 与 $\ket{\downarrow}$，其对应的特征值是 $\pm\frac{\hbar}{2}$。对于 $\hat{S}_x$ 和 $\hat{S}_y$，其特征值应该与 $\hat{S}_z$ 的相同（毕竟最初 $z$ 就是任选的），但其对应的特征矢量有所不同。以 $\hat{S}_x$ 为例，其特征矢量是：

$$
\begin{equation*}
    \ket{\uparrow}^{(x)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix} \qquad
    \ket{\downarrow}^{(x)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}} \end{pmatrix}
\end{equation*}
$$

在 $\ket{\uparrow}^{(x)}$ 和 $\ket{\downarrow}^{(x)}$ 组成的基下，任意状态 $\ket{\chi}$ 可以写成：

$$
\begin{equation*}
    \ket{\chi} = \frac{\alpha + \beta}{\sqrt{2}}\ket{\uparrow}^{(x)} + \frac{\alpha - \beta}{\sqrt{2}}\ket{\downarrow}^{(x)}
\end{equation*}
$$

从 $z$ 方向的基到 $x$ 方向的基，变换矩阵为：

$$
\begin{equation*}
    \hat{H} = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
\end{equation*}
$$

类似地，$y$ 方向的基是：

$$
\begin{equation*}
    \ket{\uparrow}^{(y)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}}i \end{pmatrix} \qquad
    \ket{\downarrow}^{(y)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}}i \end{pmatrix}
\end{equation*}
$$

任意矢量 $\ket{\chi}$ 可以写成：

$$
\begin{equation*}
    \ket{\chi} = \frac{\alpha + i\beta}{\sqrt{2}}\ket{\uparrow}^{(y)} + \frac{\alpha - i\beta}{\sqrt{2}}\ket{\downarrow}^{(y)}
\end{equation*}
$$

我们花了不小的篇幅介绍 $1/2$ 自旋粒子的这样一个两状态系统，这是因为它拥有和量子电路中的最小信息单元：量子比特完全相同的性质。事实上，利用电子自旋是制作量子计算机的一种可能性。下一章中，我们将正式进入量子电路的学习。

## 量子电路

和传统电路相似，**量子电路（Quantum Circuit）** 是由多个 **量子比特（Qubit）** 和 **量子门（Quantum Gate）** 构成的。量子比特和传统计算机中的比特类似，其确定状态有 $\ket{0}$ 和 $\ket{1}$ 两种。不过其一般状态如下所示：

$$
\begin{equation*}
    \ket{\psi} = \alpha\ket{0} + \beta\ket{1}
\end{equation*}
$$

回忆量子力学中的知识，这里的 $\ket{\psi}$ 是希尔伯特空间任取的矢量，而 $\ket{0}$ 与 $\ket{1}$ 是一组特定的特征矢量。因此当我们对如上状态的量子比特测量时，它有 $|\alpha|^2$ 的概率展示 $\ket{0}$，而 $|\beta|^2$ 的概率展示 $\ket{1}$，同时满足：

$$
\begin{equation*}
    |\alpha|^2 + |\beta|^2 = 1
\end{equation*}
$$

我们可以让 $\ket{0}$ 和 $\ket{1}$ 作为基，此时下面的矢量唯一代表了一个状态：

$$
\begin{equation*}
    \ket{\psi} = \begin{pmatrix} \alpha \\ \beta \end{pmatrix}
\end{equation*}
$$

从形式来看，这和 $1/2$ 自旋粒子的 $z$ 方向自旋的两种状态完全一致。类似地，以 $x$ 方向旋量为基时，我们可以得到：

$$
\begin{equation*}
    \ket{\psi} = \frac{\alpha + \beta}{\sqrt{2}}\ket{+} + \frac{\alpha - \beta}{\sqrt{2}}\ket{-}
\end{equation*}
$$

这里的 $\ket{\pm}$ 就是上一章中给出的 $\ket{\uparrow}^{(z)}$ 与 $\ket{\downarrow}^{(z)}$。类似地，我们可以给出以 $y$ 方向旋量为基的形式：

$$
\begin{equation*}
    \ket{\psi} = \frac{\alpha + i\beta}{\sqrt{2}}\ket{i+} + \frac{\alpha - i\beta}{\sqrt{2}}\ket{i-}
\end{equation*}
$$

量子门是接受一组量子比特输入，并输出相同数量量子比特的装置。这点和传统电路有明显的不同：后者中的许多电路门，如异或，输入两个比特但只返回一个比特。同时，我们要求所有的量子门都是幺正矩阵。这两个性质使得所有量子门都是可逆的：对于任意的输入 $\ket{\psi_{n-1}\dots\psi_0}$ 和量子门 $\hat{U}$，其输出结果 $\ket{\psi_{n-1}'\dots\psi_0'} = \hat{U}\ket{\psi_{n-1}\dots\psi_0}$ 可以再次输入同样的量子门得到：

$$
\begin{equation*}
    \hat{U}\ket{\psi_{n-1}'\dots\psi_0'} = \hat{U}\hat{U}\ket{\psi_{n-1}\dots\psi_0} = \ket{\psi_{n-1}\dots\psi_0}
\end{equation*}
$$

由于量子比特可以理解为矢量，量子门成为和矩阵等价的量；我本可以不区分两者的记号，统统使用带有尖角的大写字母表示，但一些量子门在习惯上采用多个字符表示，此时尖角记号便不适用了。对于这种特殊情形，我会使用正体字表示量子门，如 $\text{GATE}$，而其对应的矩阵会使用 $\mathcal{M}(\text{GATE})$ 表示。

下面的几个小节中，我们将介绍一些常见的量子门及其运算性质。

### 泡利门

**泡利门（The Pauli Gates）** 是自旋为 $1/2$ 的粒子的 $x$、$y$、$z$ 三方向自选分量对应的算符，在量子计算中记为 $\hat{X}$、$\hat{Y}$ 和 $\hat{Z}$。它们都接受一个量子比特，映射规则如下：

$$
\begin{equation*}
    \hat{X}: 
    \begin{cases}
        \ket{0} \mapsto \ket{1} \\
        \ket{1} \mapsto \ket{0}
    \end{cases} \quad
    \hat{Y}:
    \begin{cases}
        \ket{0} \mapsto -i\ket{1} \\
        \ket{1} \mapsto i\ket{0}
    \end{cases} \quad
    \hat{Z}:
    \begin{cases}
        \ket{0} \mapsto \ket{0} \\
        \ket{1} \mapsto -\ket{1}
    \end{cases}
\end{equation*}
$$

它们的矩阵形式可以通过矩阵元素的提取公式 $T_{ij} = \bracket{e_i}{\hat{T}}{e_j}$ 得到，以 $\hat{X}$ 为例：

$$
\begin{equation*}
    \hat{X} = \begin{pmatrix}
        \bracket{0}{\hat{X}}{0} & \bracket{0}{\hat{X}}{1} \\
        \bracket{1}{\hat{X}}{0} & \bracket{1}{\hat{X}}{1}
    \end{pmatrix} = \begin{pmatrix}
        \braket{0}{1} & \braket{0}{0} \\
        \braket{1}{1} & \braket{1}{0}
    \end{pmatrix} = \begin{pmatrix}
        0 & 1 \\ 1 & 0
    \end{pmatrix}
\end{equation*}
$$

类似地我们得到：

$$
\begin{equation*}
    \hat{Y} = \begin{pmatrix}
        0 & -i \\ i & 0
    \end{pmatrix} \qquad
    \hat{Z} = \begin{pmatrix}
        1 & 0 \\ 0 & -1
    \end{pmatrix}
\end{equation*}
$$

泡利门之间有着有趣的联系（它们是 **泡利群（Pauli Group）** 的生成元），值得一提的有：

- 每一个门都可以表示为其它两个的右手积（考虑笛卡尔系中的基矢量的叉积关系，如 $\hat{z} = \hat{x}\times\hat{y}$）乘以 $i$，即：

    $$
    \begin{equation*}
        \hat{X} = i\hat{Y}\hat{Z} \quad
        \hat{Y} = i\hat{Z}\hat{X} \quad
        \hat{Z} = i\hat{X}\hat{Y}
    \end{equation*}
    $$

- 任两个门满足反对称性：
    
    $$
    \begin{equation*}
        \hat{Y}\hat{X} = -\hat{X}\hat{Y} \quad
        \hat{Z}\hat{Y} = -\hat{Y}\hat{Z} \quad
        \hat{X}\hat{Z} = -\hat{Z}\hat{X}
    \end{equation*}
    $$
    
从基本观察中可以发现，$\hat{X}$ 将 $\ket{0}$ 和 $\ket{1}$ 的概率调转，其行为和传统电路中的 $NOT$ 比较相似，因此也被称为 **非门（Not Gate）**。此外的两个门目前看起来还比较抽象，我们会在合适的实际详细介绍它们的几何意义。

### 相位偏移门

**相位偏移门（Phase Shift Game）** 对 $\ket{1}$ 进行 $\phi$ 角度的相位偏移，其映射规则如下：

$$
\begin{equation*}
    \hat{P}(\phi):
    \begin{cases}
        \ket{0} \mapsto \ket{0} \\
        \ket{1} \mapsto e^{i\phi}\ket{1}
    \end{cases}
\end{equation*}
$$

因此它的矩阵形式为：

$$
\begin{equation*}
    \hat{P}(\phi) = \begin{pmatrix}
        1 & 0 \\ 0 & e^{i\phi}
    \end{pmatrix}
\end{equation*}
$$

相位偏移并不对原状态的概率进行改变，而仅仅改变 $\ket{1}$ 分量上的相位。当 $\phi$ 取特定值时，我们常常使用特殊的记号。比如：

- $\phi = 0$，此时正好是单位门 $\hat{I}$。
- $\phi = \frac{\pi}{4}$，此时记作 $\hat{T}$。
- $\phi = \frac{\pi}{2}$，此时记作 $\hat{S}$。
- $\phi = \pi$，此时正好是泡利门 $\hat{Z}$。

相位偏移门是斜厄米特矩阵，而非厄米特矩阵，因此它们的共轭转置和其自身不同。

### 哈达玛门

**哈达玛门（Hadamard Gate）** 将量子比特转变为一个叠加状态：

$$
\begin{equation*}
    \hat{H}: 
    \begin{cases}
        \ket{0} \mapsto \dfrac{\ket{0} + \ket{1}}{\sqrt{2}} = \ket{+} \\
        \ket{1} \mapsto \dfrac{\ket{0} - \ket{1}}{\sqrt{2}} = \ket{-}
    \end{cases}
\end{equation*}
$$

其矩阵形式是：

$$
\begin{equation*}
    \hat{H} = \frac{1}{\sqrt{2}}\begin{pmatrix}
        1 & 1 \\ 1 & -1
    \end{pmatrix}
\end{equation*}
$$

多数情况下，量子比特的初始状态都是 $\ket{0}$，此时我们可以通过哈达玛门将其变成 $\ket{0}$ 和 $\ket{1}$ 的叠加态，便于后续的操作。

前面我们使用的 $\ket{+}$ 和 $\ket{-}$ 构成了另一组常用的基。有趣的是，我们可以将它们通过哈达玛门再次转换成 $\ket{0}$ 和 $\ket{1}$ 的基：

$$
\begin{equation*}
    \hat{H}\ket{+} = \ket{0} \quad
    \hat{H}\ket{-} = \ket{1}
\end{equation*}
$$

我们将此前介绍的 $\ket{0}$ 和 $\ket{1}$（这样通过数字表示的）基称为 **计算基（Computational Basis）**，其实际相当于空间中笛卡尔系下的标准正交基。由 $\ket{+}$ 和 $\ket{-}$ 构成的基则称为 **哈达玛基（Hadamard Basis）**。今后这两种基都会被频繁用到。

### 受控非门

**受控门（Controlled Gate）** 接收两个量子比特，其中一个是 **控制量子比特（Controlled Qubit）**，当且仅当该比特中含有 $\ket{1}$ 分量时，会等概率向另一个比特执行特定操作。其中最常用的便是 **受控非门（Controlled Not Gate）**：

$$
\begin{equation*}
    \hat{CX}:
    \begin{cases}
        \ket{00} \mapsto \ket{00} \\
        \ket{01} \mapsto \ket{01} \\
        \ket{10} \mapsto \ket{11} \\
        \ket{11} \mapsto \ket{10}
    \end{cases}
\end{equation*}
$$

其矩阵形式为：

$$
\begin{equation*}
    \hat{CX} = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 1 & 0
    \end{pmatrix}
\end{equation*}
$$